In [2]:
!ls

assignment_2.ipynb  brown.txt  question.pdf


In [3]:
import string, re
from nltk.tokenize import sent_tokenize, word_tokenize
from tqdm.auto import tqdm
from collections import defaultdict
import numpy as np
from collections import Counter
import random, itertools

In [4]:
class txt_data():
    '''
    Master class to load dataset
    '''
    def __init__(self,path):
        self.path = path
        
        # list of sentences of list of tuples defining pos_tag and words.
        self.data = self.___sent_to_tags___(self.___open___(self)) 
        random.shuffle(self.data)
        self.train =  self.data[:int(len(self.data)*0.8)]
        self.val = self.data[int(len(self.data)*0.8):]
        
        self.pos_tags = [i for i in set([j[0] for i in self.data for j in i])] #list of unique pos_tags
        self.vocab = [i for i in set([j[1] for i in self.data for j in i])] # list of vocab
            
    
    def ___open___(self,path):
        '''
        To read a text file
        path: path to the text file
        '''
        with open(self.path, "r") as f:
            return f.readlines()
    
    def ___sent_to_tags___(self,sents):
        '''
        '''
        dummy = []
        for i in tqdm(sents):
            d = []
            for j in i.split():
                try: d.append((j.split('_')[1],j.split('_')[0].lower()))
                except: pass #print(f"--{i}--{j}\n")
            dummy.append(d)
        return dummy

In [5]:
path = "brown.txt"
brown = txt_data(path)
train, val = brown.train, brown.val
pos_tags, vocab = brown.pos_tags, brown.vocab

In [6]:
len(brown.data), len(train), len(val), len(train)+len(val)

(55145, 44116, 11029, 55145)

### Converitng emission/transition matrix to prob

In [7]:
def prob(matrix): 
    dummy = []
    for i in tqdm(np.array(matrix)):
        if np.sum(i) !=0: dummy.append(i/np.sum(i))
        else: dummy.append(i)
    return np.array(dummy)

### emission matrix

In [8]:
def e_matrix(data,vocab,pos_tags):
    '''
    emission matrix: count of words given the pos_tag
    data = list of sents with the tags: format is list of list of sentences with word and tag as a tuple
    vocab = list of vocab
    pos_tags = list of pos_tags
    
    return a list of matrix (pos_tags,vocab) with count(vocab/pos_tags)
    '''
    data = [i for j in data for i in j] # list of pos_tag and word pair: [('VB', 'suppose'), ('NP', 'lauren'),()]
    dummy = {} # dictioanry with tag and list of words as key, value pair, {'AT' :['the', 'the', 'a', 'the'],}
    for i in data:
        if i[0] in dummy.keys():
            dummy[i[0]].append(i[1])
        else: dummy[i[0]] = [i[1]]
    
    e_matrix = [] # emission count matrix with tag and count of words as row, column pair.
    for i in tqdm(pos_tags):
        try:
            count = Counter(dummy[i]) # count of words in a list. 
            e_matrix.append([count[j] for j in vocab])
        except: e_matrix.append([0 for j in vocab])
    
    return pos_tags, prob(e_matrix)
e_tag, e_prob = e_matrix(train, vocab, pos_tags)

In [9]:
# a = e_prob
# a [a==0] = 100*e_prob.mean()*e_prob.var()

In [10]:
count = 0
for r,i in enumerate(e_prob):
       if np.sum(i) == 0:
            print(r)
            count +=1
            
count

11
64
98
124
144
183
187
197
216
238
251
257
262
280
330
332
357
360
382
440
470


21

### transition matrix
<!-- start = Counter([j for i in data for j in i]) # starting tag count -->
<!-- t_matrix["<s>"] = [start[j] for j in pos_tags] -->

In [11]:
def t_matrix(data, pos_tags, n_gram=2):
    '''
    
    '''
    data = [[j[0]  for j in i] for i in data] # list of tags for each sentence: [['AT', 'NN', CS', '.'],[],] 
    #list of n_grams of tags, for 2 grams [['AT', 'NN'], ['NN', 'MD'], ['MD', 'HV'],[],]
    tags = [i[j:j+n_gram] for i in data for j in range(len(i) - n_gram + 1)] 
    
    # list of tags of cartesian product
    keys = [' '.join([j for j in i]) for i in tqdm(list(itertools.product(pos_tags, repeat=n_gram-1)))]
    keys.append("<s>")
    
    tags_ = {i:[] for i in keys} # initialize an empty dictionary, dict with
    for i in tags:
        a = ''
        for j in i[:-1]:
            a+= f"{j} "
        tags_[a.strip()].append(i[-1])
    tags_['<s>'] = [j for i in data for j in i]
    
    t_matrix = [] # transition count matrix with n_gram tag and count of pos_tags as key, value pair.
    for i in tqdm(keys):
        count = Counter(tags_[i])
        d = [count[j] for j in pos_tags]
        t_matrix.append(d)
        
    return keys, prob(t_matrix)
t_tag, t_prob = t_matrix(train, pos_tags, n_gram=2)

In [13]:
len(t_prob), len(t_tag)

(473, 473)

In [14]:
count = 0
for r,i in enumerate(t_prob):
       if np.sum(i) == 0:
            print(r)
            count+=1
count

11
64
98
124
144
183
187
197
216
238
251
257
262
280
330
332
357
360
382
440
470


21

In [16]:
e_prob[e_prob == 0] = 0.000000001*e_prob.mean()
t_prob[t_prob == 0] = 0.000000001*t_prob.mean()

# Viterbi algorithm

In [134]:
def viterbi(word, p_t, p_p, pos_tags, t_prob, e_prob, t_tag, e_tag,v_i):
    d_p = []
    d_t = p_t
    for r,i in enumerate(pos_tags):
        prob = 0
        tag = ''
        c_e = e_prob[t_tag.index(i)][v_i[word]]
        for j in range(len(p_t)):
            c_t = t_prob[t_tag.index(p_t[j][-1])][r]
            if c_t*c_e >= prob:
                prob = c_t
                tag = i
#             break
#         break
        d_p.append(prob)
        d_t[r].append(i)
    return d_t, d_p

In [49]:
500*500

250000

In [123]:
a = [1,2,3]

In [141]:
def viterbi(word, p_t, p_p, pos_tags, t_prob, e_prob, t_tag, e_tag,v_i):
    d_p = []
    d_t = []
    for r,i in enumerate(pos_tags):
        prob = 0.0
        tag = ''
        c_e = e_prob[e_tag.index(i)][v_i[word]]
        for j in p_t:
            c_t = t_prob[t_tag.index(j[-1])][r]
            if c_t*c_e >= prob:
                prob = c_t
                tag = i
        d_p.append(prob)
        d_t.append(tag)
    for k in range(len(pos_tags)):
        p_t[k].append(d_t[k])
#     print(p_t)
    return p_t, d_p

out = []
num_t = len(pos_tags) # number of pos tags
v_i = {i:r for r,i in enumerate(vocab)}
for i in val:
    sent = ["<s>"] + [j[-1] for j in i]
    true_tags = [j[0] for j in i]
#     print(sent, true_tags)
    dummy_beam = [([[sent[0]] for i in range(num_t)],np.ones((num_t))),] # tags,prob
    for r, word in enumerate(sent[1:]):
        if r == 5: break
        print(r)
        p_t = dummy_beam[r][0] # prev tag
        p_p = dummy_beam[r][1] # prev prob
        current_t, current_prob = viterbi(word, p_t,p_p, pos_tags, t_prob, e_prob, t_tag, e_tag, v_i)
#         break
        dummy_beam.append((current_t, current_prob))
#         print(current_t, current_prob)

    break

0
1
2
3
4


In [146]:
dummy_beam[0]

([['<s>', 'RP-HL', 'RP-HL', 'RP-HL', 'RP-HL', 'RP-HL'],
  ['<s>',
   'PPSS+MD-NC',
   'PPSS+MD-NC',
   'PPSS+MD-NC',
   'PPSS+MD-NC',
   'PPSS+MD-NC'],
  ['<s>', 'NRS', 'NRS', 'NRS', 'NRS', 'NRS'],
  ['<s>', 'WPS+BEZ', 'WPS+BEZ', 'WPS+BEZ', 'WPS+BEZ', 'WPS+BEZ'],
  ['<s>', 'VB+JJ-NC', 'VB+JJ-NC', 'VB+JJ-NC', 'VB+JJ-NC', 'VB+JJ-NC'],
  ['<s>', 'PPSS+VB', 'PPSS+VB', 'PPSS+VB', 'PPSS+VB', 'PPSS+VB'],
  ['<s>', 'CS-HL', 'CS-HL', 'CS-HL', 'CS-HL', 'CS-HL'],
  ['<s>', 'DTS+BEZ', 'DTS+BEZ', 'DTS+BEZ', 'DTS+BEZ', 'DTS+BEZ'],
  ['<s>', 'HV-NC', 'HV-NC', 'HV-NC', 'HV-NC', 'HV-NC'],
  ['<s>', 'FW-IN+NN', 'FW-IN+NN', 'FW-IN+NN', 'FW-IN+NN', 'FW-IN+NN'],
  ['<s>', 'PPSS+HV', 'PPSS+HV', 'PPSS+HV', 'PPSS+HV', 'PPSS+HV'],
  ['<s>', 'PPL-NC', 'PPL-NC', 'PPL-NC', 'PPL-NC', 'PPL-NC'],
  ['<s>', 'FW-PP$', 'FW-PP$', 'FW-PP$', 'FW-PP$', 'FW-PP$'],
  ['<s>', "'", "'", "'", "'", "'"],
  ['<s>', 'RB+BEZ-NC', 'RB+BEZ-NC', 'RB+BEZ-NC', 'RB+BEZ-NC', 'RB+BEZ-NC'],
  ['<s>', 'AP-NC', 'AP-NC', 'AP-NC', 'AP-NC', 'AP-

In [ ]:
dummy_beam[0][0]

In [ ]:
dummy['as'] = (1,1)

In [ ]:
dummy['s'] = 4

In [ ]:
dummy

# Confusion matrix

In [ ]:
expected = [1, 1, 0,2,2,2,3,3,4,0,0,4,4,4,4,4,4,4,4]
predicted = [1, 0, 0,2,1,2,3,3,3,4,3,3,4,3,4,4,4,4,4]

# expected = [1, 1, 0, 1, 0, 0, 1, 0, 0, 0]
# predicted = [1, 0, 0, 1, 0, 0, 1, 1, 1, 0]

Counter(expected)

In [ ]:
dummy = {str(i):{str(j) :0 for j in set(expected)} for i in set(expected) }
for r, i in enumerate(expected):
    dummy[str(i)][str(predicted[r])] +=1

conf_m = np.array([[j for j in i.values()] for i in dummy.values()]) # confusion matrix
dummy, conf_m

In [ ]:
pre = np.sum(conf_m,axis=0)
rec = np.sum(conf_m,axis=1)
#classwise 
precision = [i[r] for r,i in enumerate(conf_m)]/pre 
recall = [i[r] for r,i in enumerate(conf_m)]/rec 
print("--",precision, recall)

#overall macro
print("macro--",sum(precision)/ len(precision), sum(recall)/len(recall))

#overall micro
precision = sum([i[r] for r,i in enumerate(conf_m)])/np.sum(pre) 
recall = sum([i[r] for r,i in enumerate(conf_m)])/np.sum(rec)
print("micro--",precision, recall)

In [ ]:
from sklearn.metrics import confusion_matrix
results = confusion_matrix(expected, predicted)
from sklearn.metrics import precision_score, recall_score
precision_score(expected, predicted, average =None), recall_score(expected, predicted, average =None)

In [ ]:
precision_score(expected, predicted, average ="macro"), recall_score(expected, predicted, average ="macro")

In [ ]:
precision_score(expected, predicted, average ="micro"), recall_score(expected, predicted, average ="micro")